In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.mobilenet import MobileNet ,preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

### 1. Training with Categorical Classification

In [ ]:
train_datagen = ImageDataGenerator(  preprocessing_function = preprocess_input).flow_from_directory(
                                        '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train', 
                                        target_size = (224, 224), batch_size = 256)

valid_datagen = ImageDataGenerator( preprocessing_function = preprocess_input).flow_from_directory(
                                    '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Test', 
                                    target_size = (224, 224), batch_size = 256)

test_datagen = ImageDataGenerator( preprocessing_function = preprocess_input).flow_from_directory(
                                    '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Validation',
                                    target_size = (224, 224), batch_size = 256)

In [ ]:
mobilenet = MobileNet(include_top = False)
x = mobilenet.output

x = GlobalAveragePooling2D()(x) 
x = Dense(1024,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512,activation='relu')(x)

preds = Dense(2,activation='softmax')(x)

model = Model(inputs = mobilenet.input, outputs = preds)

for layer in model.layers[:-9]:
    layer.trainable = False

model.compile( optimizer = 'adam',  loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
filepath = "saved-model-{epoch:02d}.keras"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='max')

In [ ]:
history = model.fit(
    train_datagen,
    steps_per_epoch=train_datagen.samples // train_datagen.batch_size,
    validation_data=valid_datagen,
    validation_steps=valid_datagen.samples // valid_datagen.batch_size,
    callbacks=[checkpoint],
    epochs=10)

### 2. Training with Binary Classification

In [ ]:
train_datagen = ImageDataGenerator(  preprocessing_function = preprocess_input).flow_from_directory(
                                        '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train', 
                                        target_size = (224, 224), batch_size = 256, class_mode = 'binary')

valid_datagen = ImageDataGenerator( preprocessing_function = preprocess_input).flow_from_directory(
                                    '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Test', 
                                    target_size = (224, 224), batch_size = 256, class_mode = 'binary')

test_datagen = ImageDataGenerator( preprocessing_function = preprocess_input).flow_from_directory(
                                    '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Validation',
                                    target_size = (224, 224), batch_size = 256, class_mode = 'binary')

In [ ]:
mobilenet = MobileNet(include_top = False)
x = mobilenet.output

x = GlobalAveragePooling2D()(x) 
x = Dense(1024,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512,activation='relu')(x)

preds = Dense(1,activation='sigmoid')(x)

model_2 = Model(inputs = mobilenet.input, outputs = preds)

for layer in model.layers[:-9]:
    layer.trainable = False

model_2.compile( optimizer = 'adam',  loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
filepath = "saved{epoch:02d}.keras"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='max')

In [ ]:
history = model_2.fit(
    train_datagen,
    steps_per_epoch=train_datagen.samples // train_datagen.batch_size,
    validation_data=valid_datagen,
    validation_steps=valid_datagen.samples // valid_datagen.batch_size,
    callbacks=[checkpoint],
    epochs=10)

### 3. Finding best Model

In [ ]:
model_2.evaluate(test_datagen)

In [ ]:
model_2.evaluate(valid_datagen)

### 4. Getting Inference

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/saved10.keras")

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to match model input
    img_array = image.img_to_array(img)  # Convert to NumPy array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch processing
    img_array = preprocess_input(img_array)  # Apply MobileNet preprocessing

    # Get prediction
    prediction = model.predict(img_array)[0][0]  # Since it's binary classification
    
    # Convert prediction to class label
    class_label = "Male" if prediction > 0.5 else "Female"

    # Show image with prediction
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Predicted: {class_label} ({prediction:.2f})")
    plt.show()

    return class_label

path = '/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train/Male/'
image_path = path + os.listdir('/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train/Male')[0]

predicted_label = predict_image(image_path)
print(f"Predicted Class: {predicted_label}")

In [ ]:
model = tf.keras.models.load_model("/kaggle/working/saved10.keras")

male_path = "/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train/Male/"
female_path = "/kaggle/input/gender-recognition-200k-images-celeba/Dataset/Train/Female/"

male_images = random.sample(os.listdir(male_path), 25)
female_images = random.sample(os.listdir(female_path), 25)

all_images = [(male_path + img, "Male") for img in male_images] + [(female_path + img, "Female") for img in female_images]
random.shuffle(all_images)

def predict_image(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    prediction = model.predict(img_array, verbose = 0)[0][0]  # Since it's binary classification
    class_label = "Male" if prediction > 0.5 else "Female"
    
    return img, class_label, prediction  # Return image object, label, and confidence score

# Create 10x5 subplot
fig, axes = plt.subplots(10, 5, figsize=(15, 30))

for i, (img_path, true_label) in enumerate(all_images):
    img, predicted_label, confidence = predict_image(img_path)
    ax = axes[i // 5, i % 5]  # Calculate subplot position
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(f"P: {predicted_label} ({confidence:.2f})\nT: {true_label}", fontsize=10)

plt.tight_layout()
plt.show()